In [1]:
from google.cloud import bigquery
import pandas as pd
import datetime
import os
from gtts import gTTS
from playsound import playsound

pd.options.display.float_format = '{:,.0f}'.format
pd.options.display.max_colwidth = 100
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

# Change this line to point to your own service token file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/dicharryd/service_token.json"

#********************************************************************
#Connecting to BigQuery
client = bigquery.Client()

def query_calls(client, query):
    job_config = bigquery.QueryJobConfig()
    job_config.use_legacy_sql = False
    query_job = client.query(query, job_config=job_config)
    results = query_job.result()
    return results

def job_setup(client, query, dataset, table, write_disposition):
    job_config = bigquery.QueryJobConfig()
    table_ref = client.dataset(dataset).table(table)
    job_config.destination = table_ref
    job_config.use_legacy_sql = False
    job_config.write_disposition = write_disposition
   
    query_job = client.query(
    query,
    location='US',
    job_config=job_config)

    query_job.result()
    print('Query results loaded to table {}'.format(table_ref.path))
    print('Query last completed at {}'.format(datetime.datetime.now()))
    last_modified = client.get_table(dataset + '.' + table).modified
    print('Table last modified at {} GMT timezone'.format(last_modified))


//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/dicharryd/service_token.json',
                                                               scope
                                                              )
# Authorise your Notebook
gc = gspread.authorize(credentials)

# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1fgIwUMo0Z9PG-gkGvFExBIVFQRNOlkFNyM6BtzTrh3w'

# PULL IN NON-SUB TRAFFIC

In [3]:
query = '''

WITH articles AS (SELECT
                  DISTINCT
                  visit_referrer,
                  timestamp(date_time) as read_time,
                  IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
                  IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
                  IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
                  IF(post_prop27 IN ('WSJ_sub_yes'), 1, 0) as sub_status,
                  CONCAT(post_visid_high, post_visid_low) as unique_id,
                  visit_num,
                  IF(prop1 = 'Article', 1, 0) as article,
                  visit_page_num,
                  IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
                  IF(page_event_var2 = 'WSJ_Article_social_share', 1, 0) as share,
                  prop10
                  FROM
                   `djomniture.cipomniture_djglobal.*` 
                  WHERE REGEXP_CONTAINS(_TABLE_SUFFIX,r'^\d{4}$*')
                  AND DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-11-01" AND "2021-11-30"
                  AND SAFE_CAST(exclude_hit AS INT64) <= 0
                  AND hit_source NOT IN ('5',
                                         '8',
                                         '9')
                  AND (REGEXP_CONTAINS(page_url, 'wsj[.com|.net]')
                  OR channel IN ('Online Journal'))),
sales as (SELECT 
          CONCAT(post_visid_high, post_visid_low) as unique_id,
          timestamp(date_time) as subscribe_time,
          post_evar39 as conversion_headline,
          visit_num,
          post_purchaseid,
          FROM
          `djomniture.cipomniture_djglobal.*`
          WHERE
          REGEXP_CONTAINS(_TABLE_SUFFIX,
                           r'^\d{4}$*')
          AND SAFE_CAST(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01') AS DATE) BETWEEN "2021-11-01" AND "2021-11-30"
          AND SAFE_CAST(exclude_hit AS INT64) <= 0
          AND hit_source NOT IN ('5',
                                 '8',
                                 '9')
          AND REGEXP_CONTAINS(post_event_list, r'^1,|,1,|,1$')
          --Relevant order filters go here (i.e. excluding student)
          AND REGEXP_CONTAINS(page_url, 'store.wsj.com|buy.wsj.com')
          AND channel IN ('Online Journal')),
performance as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','WSJ_wsj_square','_wsj_square') THEN prop10
WHEN news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'wsj.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT articles.unique_id) as uniques,
COUNT(articles.views) as views,
COUNT(DISTINCT CONCAT(articles.unique_id, articles.visit_num)) as visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >1, CONCAT(articles.unique_id), NULL)) as twoplus,
COUNT(DISTINCT IF(articles.visit_num = '1', CONCAT(articles.unique_id, articles.visit_num), NULL)) as cold_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) BETWEEN 2 AND 4, CONCAT(articles.unique_id, articles.visit_num), NULL)) as medium_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >= 5, CONCAT(articles.unique_id, articles.visit_num), NULL)) as hot_visits,
COUNT(DISTINCT IF(articles.share = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as shares,
COUNT(DISTINCT IF(articles.visit_num = sales.visit_num, post_purchaseid, null)) as orders_same_visit,
COUNT(DISTINCT post_purchaseid) as orders_7_days,
COUNT(DISTINCT post_purchaseid) as purchases,
COUNT(DISTINCT articles.unique_id) as unique_ids,
COUNT(DISTINCT post_purchaseid)  / COUNT(DISTINCT articles.unique_id) as conversion_rate,
SAFE_DIVIDE(COUNT(DISTINCT IF(article = 1 AND locked = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)), COUNT(DISTINCT IF(article = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL))) as percent_locked,
COUNT(DISTINCT IF(article = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as all_articles,
COUNT(DISTINCT IF(article = 1 AND locked = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as locked_articles
FROM
articles
LEFT JOIN
sales ON 
articles.unique_id = sales.unique_id
AND articles.read_time < sales.subscribe_time
AND DATE_DIFF(DATE(sales.subscribe_time), DATE(articles.read_time), DAY) < 7
WHERE articles.sub_status = 0
AND articles.article_visit = 1
GROUP BY
referrer,
month),
return_visits as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','WSJ_wsj_square','_wsj_square') THEN articles.prop10
WHEN articles.news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'wsj.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM articles.read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT IF(returns.read_time > articles.read_time AND DATE_DIFF(DATE(returns.read_time), DATE(articles.read_time), DAY) < 7
AND SAFE_CAST(returns.visit_num as INT64) > 1, CONCAT(articles.unique_id, articles.visit_num), NULL)) as returns
FROM
articles
LEFT JOIN
articles as returns ON 
articles.unique_id = returns.unique_id
WHERE articles.sub_status = 0
AND articles.article_visit = 1
AND articles.visit_num = '1'
GROUP BY
referrer, month)
SELECT
*,
SAFE_DIVIDE(twoplus, uniques) as return_rate
FROM
performance
LEFT JOIN
return_visits
USING(referrer, month)
WHERE uniques >= 1000
ORDER BY
uniques DESC

'''

wsjreferral = query_calls(client, query).to_dataframe()

In [4]:
wsjreferral['groupreferrers'] = pd.np.where(wsjreferral['referrer'].str.contains("goog"),"Google",
                          pd.np.where(wsjreferral['referrer'].str.contains("android.gm"),"Google",
                                     pd.np.where(wsjreferral['referrer']=="facebook.com","Facebook",
                                         pd.np.where(wsjreferral['referrer']=="News Tab","FB News",
                             pd.np.where(wsjreferral['referrer'].str.contains("twitter"),"Twitter",
                                             pd.np.where(wsjreferral['referrer']=="t.co","Twitter",
                                pd.np.where(wsjreferral['referrer'].str.contains("lnkd"),"LinkedIn",
                            pd.np.where(wsjreferral['referrer'].str.contains("linkedin"),"LinkedIn",
                                     pd.np.where(wsjreferral['referrer']=="linkedin.com","LinkedIn",
                               pd.np.where(wsjreferral['referrer'].str.contains("yahoo"),"Yahoo",
                                          pd.np.where(wsjreferral['referrer']=="wsj.com","Direct",
                           pd.np.where(wsjreferral['referrer'].str.contains("robinhood"),"Robinhood",
                             pd.np.where(wsjreferral['referrer'].str.contains("wsj_RHF"),"Robinhood",
                                       pd.np.where(wsjreferral['referrer']=="apple.news","Apple News",
                                                                             wsjreferral['referrer']))))))))))))))
wsjreferral = wsjreferral.groupby('groupreferrers').sum().sort_values(by='uniques',ascending=False).reset_index()

wsjreferral['return_rate'] = ((wsjreferral['twoplus'] / wsjreferral['uniques']))
wsjreferral['new_percent_locked'] = ((wsjreferral['locked_articles'] / wsjreferral['all_articles']))
wsjreferral['new_conversion_rate'] = ((wsjreferral['purchases'] / wsjreferral['unique_ids']))

wsjreferral['new_percent_locked'] = wsjreferral['new_percent_locked'].map("{:.2}".format)
wsjreferral['new_conversion_rate'] = wsjreferral['new_conversion_rate'].map("{:.3}".format)
wsjreferral.insert(0, 'brand', 'wsj')

mytext = 'WSJ'
language = 'en'
job = gTTS(text=mytext, lang=language, slow=False)
job.save("wsj.mp3")
playsound('wsj.mp3')

#wsjreferral

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-pa

In [5]:
pd.set_option('display.max_columns', 100)
wsjreferral

,brand,groupreferrers,uniques,views,visits,twoplus,cold_visits,medium_visits,hot_visits,shares,orders_same_visit,orders_7_days,purchases,unique_ids,conversion_rate,percent_locked,all_articles,locked_articles,returns,return_rate,new_percent_locked,new_conversion_rate
0,wsj,Google,26137157,34283015,28296667,3239573,23267266,2005789,3023612,62516,7840,10220,10220,26137157,0,8,31808748,8637381,412788,0,0.27,0.000391
1,wsj,Direct,18618391,35111204,22002611,3062313,16199215,2100202,3703194,298009,9411,12971,12971,18618391,0,0,29559545,10719432,797920,0,0.36,0.000697
2,wsj,Facebook,4693632,7926716,5458223,1259344,3716402,1062837,678984,27328,1765,2221,2221,4693632,0,0,6853313,2519852,297263,0,0.37,0.000473
3,wsj,Twitter,2197944,3881591,2556699,577644,1753366,397501,405832,17488,977,1255,1255,2197944,0,1,3496640,1310324,152183,0,0.37,0.000571
4,wsj,FB News,1495339,2264253,1759156,348870,1258835,331446,168875,4574,67,87,87,1495339,0,0,2220622,473042,117135,0,0.21,5.82e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,wsj,padlet.com,1056,1238,1065,42,1016,29,20,4,0,0,0,1056,0,0,1141,151,3,0,0.13,0.0
140,wsj,foxbusiness.com,1047,2124,1068,277,777,153,138,24,3,3,3,1047,0,1,1661,1507,33,0,0.91,0.00287
141,wsj,quillette.com,1040,2203,1103,424,640,225,238,19,1,2,2,1040,0,1,1676,1563,65,0,0.93,0.00192
142,wsj,nytimes.com,1040,2111,1095,353,709,179,207,6,4,5,5,1040,0,1,1585,797,56,0,0.5,0.00481


# BARRONS

In [6]:
query = '''

WITH articles AS (SELECT
                  DISTINCT
                  visit_referrer,
                  timestamp(date_time) as read_time,
                  IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
                  IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
                  IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
                  IF(post_prop27 IN ('Barrons Online_sub_yes', 'BOL_sub_yes'), 1, 0) as sub_status,
                  CONCAT(post_visid_high, post_visid_low) as unique_id,
                  visit_num,
                  IF(prop1 = 'Article', 1, 0) as article,
                  visit_page_num,
                  IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
                  IF(page_event_var2 = 'BOL_Article_social_share', 1, 0) as share,
                  prop10
                  FROM
                   `djomniture.cipomniture_djglobal.*` 
                  WHERE REGEXP_CONTAINS(_TABLE_SUFFIX,r'^\d{4}$*')
                  AND DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-11-01" AND "2021-11-30"
                  AND SAFE_CAST(exclude_hit AS INT64) <= 0
                  AND hit_source NOT IN ('5',
                                         '8',
                                         '9')
                  AND (REGEXP_CONTAINS(page_url, 'barrons[.com|.net]')
                  OR channel IN ('Barrons Online', 'Barrons'))),
sales as (SELECT 
          CONCAT(post_visid_high, post_visid_low) as unique_id,
          timestamp(date_time) as subscribe_time,
          post_evar39 as conversion_headline,
          visit_num,
          post_purchaseid,
          FROM
          `djomniture.cipomniture_djglobal.*`
          WHERE
          REGEXP_CONTAINS(_TABLE_SUFFIX,
                           r'^\d{4}$*')
          AND SAFE_CAST(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01') AS DATE) BETWEEN "2021-11-01" AND "2021-11-30"
          AND SAFE_CAST(exclude_hit AS INT64) <= 0
          AND hit_source NOT IN ('5',
                                 '8',
                                 '9')
          AND REGEXP_CONTAINS(post_event_list, r'^1,|,1,|,1$')
          --Relevant order filters go here (i.e. excluding student)
          AND REGEXP_CONTAINS(page_url, 'store.barrons.com|buy.barrons.com')
          AND channel IN ('Barrons Online', 'Barrons')),
performance as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','BOL_bol_square','_bol_square') THEN prop10
WHEN news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'barrons.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT articles.unique_id) as uniques,
COUNT(articles.views) as views,
COUNT(DISTINCT CONCAT(articles.unique_id, articles.visit_num)) as visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >1, CONCAT(articles.unique_id), NULL)) as twoplus,
COUNT(DISTINCT IF(articles.visit_num = '1', CONCAT(articles.unique_id, articles.visit_num), NULL)) as cold_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) BETWEEN 2 AND 4, CONCAT(articles.unique_id, articles.visit_num), NULL)) as medium_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >= 5, CONCAT(articles.unique_id, articles.visit_num), NULL)) as hot_visits,
COUNT(DISTINCT IF(articles.share = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as shares,
COUNT(DISTINCT IF(articles.visit_num = sales.visit_num, post_purchaseid, null)) as orders_same_visit,
COUNT(DISTINCT post_purchaseid) as orders_7_days,
COUNT(DISTINCT post_purchaseid) as purchases,
COUNT(DISTINCT articles.unique_id) as unique_ids,
COUNT(DISTINCT post_purchaseid)  / COUNT(DISTINCT articles.unique_id) as conversion_rate,
SAFE_DIVIDE(COUNT(DISTINCT IF(article = 1 AND locked = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)), COUNT(DISTINCT IF(article = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL))) as percent_locked,
COUNT(DISTINCT IF(article = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as all_articles,
COUNT(DISTINCT IF(article = 1 AND locked = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as locked_articles
FROM
articles
LEFT JOIN
sales ON 
articles.unique_id = sales.unique_id
AND articles.read_time < sales.subscribe_time
AND DATE_DIFF(DATE(sales.subscribe_time), DATE(articles.read_time), DAY) < 7
WHERE articles.sub_status = 0
AND articles.article_visit = 1
GROUP BY
referrer,
month),
return_visits as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','BOL_bol_square','_bol_square') THEN articles.prop10
WHEN articles.news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'barrons.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM articles.read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT IF(returns.read_time > articles.read_time AND DATE_DIFF(DATE(returns.read_time), DATE(articles.read_time), DAY) < 7
AND SAFE_CAST(returns.visit_num as INT64) > 1, CONCAT(articles.unique_id, articles.visit_num), NULL)) as returns
FROM
articles
LEFT JOIN
articles as returns ON 
articles.unique_id = returns.unique_id
WHERE articles.sub_status = 0
AND articles.article_visit = 1
AND articles.visit_num = '1'
GROUP BY
referrer, month)
SELECT
*,
SAFE_DIVIDE(twoplus, uniques) as return_rate
FROM
performance
LEFT JOIN
return_visits
USING(referrer, month)
WHERE uniques >= 1000
ORDER BY
uniques DESC

'''

barreferral = query_calls(client, query).to_dataframe()

In [7]:
barreferral['groupreferrers'] = pd.np.where(barreferral['referrer'].str.contains("goog"),"Google",
                          pd.np.where(barreferral['referrer'].str.contains("android.gm"),"Google",
                                     pd.np.where(barreferral['referrer']=="facebook.com","Facebook",
                                         pd.np.where(barreferral['referrer']=="News Tab","FB News",
                             pd.np.where(barreferral['referrer'].str.contains("twitter"),"Twitter",
                                             pd.np.where(barreferral['referrer']=="t.co","Twitter",
                                pd.np.where(barreferral['referrer'].str.contains("lnkd"),"LinkedIn",
                            pd.np.where(barreferral['referrer'].str.contains("linkedin"),"LinkedIn",
                                     pd.np.where(barreferral['referrer']=="linkedin.com","LinkedIn",
                               pd.np.where(barreferral['referrer'].str.contains("yahoo"),"Yahoo",
                             pd.np.where(barreferral['referrer'].str.contains("barrons"),"Direct",
                           pd.np.where(barreferral['referrer'].str.contains("robinhood"),"Robinhood",
                             pd.np.where(barreferral['referrer'].str.contains("bar_RHF"),"Robinhood",
                                       pd.np.where(barreferral['referrer']=="apple.news","Apple News",
                                                                             barreferral['referrer']))))))))))))))
barreferral = barreferral.groupby('groupreferrers').sum().sort_values(by='uniques',ascending=False).reset_index()

barreferral['return_rate'] = ((barreferral['twoplus'] / barreferral['uniques']))
barreferral['new_percent_locked'] = ((barreferral['locked_articles'] / barreferral['all_articles']))
barreferral['new_conversion_rate'] = ((barreferral['purchases'] / barreferral['unique_ids']))

barreferral['new_percent_locked'] = barreferral['new_percent_locked'].map("{:.2}".format)
barreferral['new_conversion_rate'] = barreferral['new_conversion_rate'].map("{:.3}".format)
barreferral.insert(0, 'brand', 'barrons')

mytext = 'barrons'
language = 'en'
job = gTTS(text=mytext, lang=language, slow=False)
job.save("barrons.mp3")
playsound('barrons.mp3')

#barreferral

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-pa

# MARKETWATCH

In [8]:
query = '''

WITH articles AS (SELECT
                  DISTINCT
                  visit_referrer,
                  timestamp(date_time) as read_time,
                  IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
                  IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
                  IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
                  IF(post_prop27 IN ('Marketwatch_sub_yes','MW_sub_yes'), 1, 0) as sub_status,
                  CONCAT(post_visid_high, post_visid_low) as unique_id,
                  visit_num,
                  IF(prop1 = 'Article', 1, 0) as article,
                  visit_page_num,
                  IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
                  IF(page_event_var2 = 'MW_Article_social_share', 1, 0) as share,
                  prop10
                  FROM
                   `djomniture.cipomniture_djglobal.*` 
                  WHERE REGEXP_CONTAINS(_TABLE_SUFFIX,r'^\d{4}$*')
                  AND DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-11-01" AND "2021-11-30"
                  AND SAFE_CAST(exclude_hit AS INT64) <= 0
                  AND hit_source NOT IN ('5',
                                         '8',
                                         '9')
                  AND (REGEXP_CONTAINS(page_url, 'marketwatch[.com|.net]')
                  OR channel IN ('MarketWatch','Marketwatch'))),
sales as (SELECT 
          CONCAT(post_visid_high, post_visid_low) as unique_id,
          timestamp(date_time) as subscribe_time,
          post_evar39 as conversion_headline,
          visit_num,
          post_purchaseid,
          FROM
          `djomniture.cipomniture_djglobal.*`
          WHERE
          REGEXP_CONTAINS(_TABLE_SUFFIX,
                           r'^\d{4}$*')
          AND SAFE_CAST(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01') AS DATE) BETWEEN "2021-11-01" AND "2021-11-30"
          AND SAFE_CAST(exclude_hit AS INT64) <= 0
          AND hit_source NOT IN ('5',
                                 '8',
                                 '9')
          AND REGEXP_CONTAINS(post_event_list, r'^1,|,1,|,1$')
          --Relevant order filters go here (i.e. excluding student)
          AND REGEXP_CONTAINS(page_url, 'store.marketwatch.com|buy.marketwatch.com')
          AND channel IN ('MarketWatch','Marketwatch')),
performance as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','MW_mw_square','_mw_square') THEN prop10 
WHEN news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'marketwatch.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT articles.unique_id) as uniques,
COUNT(articles.views) as views,
COUNT(DISTINCT CONCAT(articles.unique_id, articles.visit_num)) as visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >1, CONCAT(articles.unique_id), NULL)) as twoplus,
COUNT(DISTINCT IF(articles.visit_num = '1', CONCAT(articles.unique_id, articles.visit_num), NULL)) as cold_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) BETWEEN 2 AND 4, CONCAT(articles.unique_id, articles.visit_num), NULL)) as medium_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >= 5, CONCAT(articles.unique_id, articles.visit_num), NULL)) as hot_visits,
COUNT(DISTINCT IF(articles.share = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as shares,
COUNT(DISTINCT IF(articles.visit_num = sales.visit_num, post_purchaseid, null)) as orders_same_visit,
COUNT(DISTINCT post_purchaseid) as orders_7_days,
COUNT(DISTINCT post_purchaseid) as purchases,
COUNT(DISTINCT articles.unique_id) as unique_ids,
COUNT(DISTINCT post_purchaseid)  / COUNT(DISTINCT articles.unique_id) as conversion_rate,
SAFE_DIVIDE(COUNT(DISTINCT IF(article = 1 AND locked = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)), COUNT(DISTINCT IF(article = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL))) as percent_locked,
COUNT(DISTINCT IF(article = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as all_articles,
COUNT(DISTINCT IF(article = 1 AND locked = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as locked_articles
FROM
articles
LEFT JOIN
sales ON 
articles.unique_id = sales.unique_id
AND articles.read_time < sales.subscribe_time
AND DATE_DIFF(DATE(sales.subscribe_time), DATE(articles.read_time), DAY) < 7
WHERE articles.sub_status = 0
AND articles.article_visit = 1
GROUP BY
referrer,
month),
return_visits as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','MW_mw_square','_mw_square') THEN articles.prop10
WHEN articles.news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'marketwatch.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM articles.read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT IF(returns.read_time > articles.read_time AND DATE_DIFF(DATE(returns.read_time), DATE(articles.read_time), DAY) < 7
AND SAFE_CAST(returns.visit_num as INT64) > 1, CONCAT(articles.unique_id, articles.visit_num), NULL)) as returns
FROM
articles
LEFT JOIN
articles as returns ON 
articles.unique_id = returns.unique_id
WHERE articles.sub_status = 0
AND articles.article_visit = 1
AND articles.visit_num = '1'
GROUP BY
referrer, month)
SELECT
*,
SAFE_DIVIDE(twoplus, uniques) as return_rate
FROM
performance
LEFT JOIN
return_visits
USING(referrer, month)
WHERE uniques >= 1000
ORDER BY
uniques DESC

'''

mwreferral = query_calls(client, query).to_dataframe()

In [9]:
mwreferral['groupreferrers'] =    pd.np.where(mwreferral['referrer'].str.contains("goog"),"Google",
                            pd.np.where(mwreferral['referrer'].str.contains("android.gm"),"Google",
                                       pd.np.where(mwreferral['referrer']=="facebook.com","Facebook",
                                           pd.np.where(mwreferral['referrer']=="News Tab","FB News",
                               pd.np.where(mwreferral['referrer'].str.contains("twitter"),"Twitter",
                                               pd.np.where(mwreferral['referrer']=="t.co","Twitter",
                                  pd.np.where(mwreferral['referrer'].str.contains("lnkd"),"LinkedIn",
                                       pd.np.where(mwreferral['referrer']=="linkedin.com","LinkedIn",
                                 pd.np.where(mwreferral['referrer'].str.contains("yahoo"),"Yahoo",
                           pd.np.where(mwreferral['referrer'].str.contains("marketwatch"),"Direct",
                             pd.np.where(mwreferral['referrer'].str.contains("robinhood"),"Robinhood",
                                pd.np.where(mwreferral['referrer'].str.contains("mw_RHF"),"Robinhood",            
                                         pd.np.where(mwreferral['referrer']=="apple.news","Apple News",
                             pd.np.where(mwreferral['referrer'].str.contains("mw_square"),"Cash App",          
                                                                               mwreferral['referrer']))))))))))))))
mwreferral = mwreferral.groupby('groupreferrers').sum().sort_values(by='uniques',ascending=False).reset_index()

mwreferral['return_rate'] = ((mwreferral['twoplus'] / mwreferral['uniques']))
mwreferral['new_percent_locked'] = ((mwreferral['locked_articles'] / mwreferral['all_articles']))
mwreferral['new_conversion_rate'] = ((mwreferral['purchases'] / mwreferral['unique_ids']))

mwreferral['new_percent_locked'] = mwreferral['new_percent_locked'].map("{:.2}".format)
mwreferral['new_conversion_rate'] = mwreferral['new_conversion_rate'].map("{:.3}".format)
mwreferral.insert(0, 'brand', 'marketwatch')

mytext = 'marketwatch'
language = 'en'
job = gTTS(text=mytext, lang=language, slow=False)
job.save("mw.mp3")
playsound('mw.mp3')

#mwreferral

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-pa

In [10]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)
combined = pd.concat([wsjreferral,barreferral,mwreferral], sort=False).reset_index()
combined['rank'] = pd.np.where(combined['groupreferrers'] == "Google",4, 
                   pd.np.where(combined['groupreferrers'] == "Facebook",3,
                   pd.np.where(combined['groupreferrers'] == "FB News",2,
                   pd.np.where(combined['groupreferrers'] == "Apple News",1,""))))                            
#combined

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  import sys


In [11]:
mytext = 'ranked'
language = 'en'
job = gTTS(text=mytext, lang=language, slow=False)
job.save("ranked.mp3")
playsound('ranked.mp3')

In [12]:
# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = '4-8 UNIQUES'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(combined,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

mytext = 'sheet updated successfully'
language = 'en'
job = gTTS(text=mytext, lang=language, slow=False)
job.save("sheetupdated.mp3")
playsound('sheetupdated.mp3')
#if you get a boolean error (truth value of a series in ambiguous), you just need to reset_index

The sheet is updated successfully
